In [22]:
#kernel = use CMIP6 AWS
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline
from scipy import stats
import matplotlib
import matplotlib.animation as animation
import cartopy
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import warnings
import matplotlib.path as mpath
import intake
import pandas as pd
import s3fs
import proplot as pplt
from numba import jit
import os
warnings.filterwarnings("ignore")

In [2]:
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')
#query = dict(variable_id = ["rsds","rsus","rlds","rlus","hfss","hfls"], table_id = "Amon",\
#             experiment_id = ['historical', 'ssp585'])
query = dict(variable_id = ["rsds","rlds","rsus","rlus","hfss","hfls"],table_id="Amon",\
             experiment_id = ['piControl', 'abrupt-4xCO2'])
res = catalog.search(require_all_on=["source_id"],**query)
models = res.df['source_id'].unique()
print((res.df['source_id'].unique()))
#Other msft* variables aren't available, so this is probably the dataset to use.
#Decent collection of models

['ACCESS-CM2' 'ACCESS-ESM1-5' 'AWI-CM-1-1-MR' 'BCC-CSM2-MR' 'BCC-ESM1'
 'CAMS-CSM1-0' 'CAS-ESM2-0' 'CESM2' 'CESM2-FV2' 'CESM2-WACCM'
 'CESM2-WACCM-FV2' 'CMCC-CM2-SR5' 'CNRM-CM6-1' 'CNRM-CM6-1-HR'
 'CNRM-ESM2-1' 'CanESM5' 'E3SM-1-0' 'FGOALS-f3-L' 'FGOALS-g3'
 'FIO-ESM-2-0' 'GFDL-CM4' 'GFDL-ESM4' 'GISS-E2-1-G' 'GISS-E2-1-H'
 'GISS-E2-2-G' 'HadGEM3-GC31-LL' 'HadGEM3-GC31-MM' 'IITM-ESM' 'INM-CM4-8'
 'INM-CM5-0' 'IPSL-CM6A-LR' 'KACE-1-0-G' 'MIROC-ES2L' 'MIROC6'
 'MPI-ESM-1-2-HAM' 'MPI-ESM1-2-HR' 'MPI-ESM1-2-LR' 'MRI-ESM2-0' 'NorCPM1'
 'NorESM2-MM' 'SAM0-UNICON' 'TaiESM1']


In [3]:
#set up dictionaries
nyr = 150
ntime = nyr*12
print(len(models))
models0 = np.delete(models,np.where(models=='KACE-1-0-G'))
print(models0)
models = models0

42
['ACCESS-CM2' 'ACCESS-ESM1-5' 'AWI-CM-1-1-MR' 'BCC-CSM2-MR' 'BCC-ESM1'
 'CAMS-CSM1-0' 'CAS-ESM2-0' 'CESM2' 'CESM2-FV2' 'CESM2-WACCM'
 'CESM2-WACCM-FV2' 'CMCC-CM2-SR5' 'CNRM-CM6-1' 'CNRM-CM6-1-HR'
 'CNRM-ESM2-1' 'CanESM5' 'E3SM-1-0' 'FGOALS-f3-L' 'FGOALS-g3'
 'FIO-ESM-2-0' 'GFDL-CM4' 'GFDL-ESM4' 'GISS-E2-1-G' 'GISS-E2-1-H'
 'GISS-E2-2-G' 'HadGEM3-GC31-LL' 'HadGEM3-GC31-MM' 'IITM-ESM' 'INM-CM4-8'
 'INM-CM5-0' 'IPSL-CM6A-LR' 'MIROC-ES2L' 'MIROC6' 'MPI-ESM-1-2-HAM'
 'MPI-ESM1-2-HR' 'MPI-ESM1-2-LR' 'MRI-ESM2-0' 'NorCPM1' 'NorESM2-MM'
 'SAM0-UNICON' 'TaiESM1']


In [16]:
#for same set of models, create time series of psl, hfss, hfls, tas as [model, time, lat, lon]
#note that longitude dimension needs to be in terms of [-180,180]
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')

dict_piC = {}

lats = {}
lons = {}

#set up AWS filesystem access
fs_s3 = s3fs.S3FileSystem(anon=True)
#myvars = ["hfls"]#,"hfss","tas","psl"]
#myvars = ["hfss"]
#myvars=["hfls","hfss"]
myvars = ["tas","psl"]
for imodel in enumerate(models):
    print("piControl, "+imodel[1])
    for ivar in range(0,len(myvars)):
        res = catalog.search(source_id=imodel[1], variable_id=myvars[ivar],\
                             experiment_id="piControl", table_id="Amon")
        #print(res.df['zstore'][0])
        mapper = fs_s3.get_mapper(res.df['zstore'][0])
        ds = xr.open_zarr(mapper, consolidated=True)
        if ivar==0:
            try:
                lats[imodel[1]] = ds.lat
                lons[imodel[1]] = ds.lon
            except AttributeError:
                ds = ds.rename({"latitude":"lat", "longitude":"lon"})
                lats[imodel[1]] = ds.lat
                lons[imodel[1]] = ds.lon
        try:
            dict_piC[imodel[1]+"_"+myvars[ivar]]=ds[myvars[ivar]]
        except (AttributeError, KeyError):
            print("Not enough years")
            ds = ds.rename({"latitude":"lat", "longitude":"lon"})
            dict_piC[imodel[1]+"_"+myvars[ivar]]=ds[myvars[ivar]]

piControl, ACCESS-CM2
piControl, ACCESS-ESM1-5
piControl, AWI-CM-1-1-MR
piControl, BCC-CSM2-MR
piControl, BCC-ESM1
piControl, CAMS-CSM1-0
piControl, CAS-ESM2-0
piControl, CESM2
piControl, CESM2-FV2
piControl, CESM2-WACCM
piControl, CESM2-WACCM-FV2
piControl, CMCC-CM2-SR5
piControl, CNRM-CM6-1
piControl, CNRM-CM6-1-HR
piControl, CNRM-ESM2-1
piControl, CanESM5
piControl, E3SM-1-0
piControl, FGOALS-f3-L
piControl, FGOALS-g3
piControl, FIO-ESM-2-0
piControl, GFDL-CM4
piControl, GFDL-ESM4
piControl, GISS-E2-1-G
piControl, GISS-E2-1-H
piControl, GISS-E2-2-G
piControl, HadGEM3-GC31-LL
piControl, HadGEM3-GC31-MM
piControl, IITM-ESM
piControl, INM-CM4-8
piControl, INM-CM5-0
piControl, IPSL-CM6A-LR
piControl, MIROC-ES2L
piControl, MIROC6
piControl, MPI-ESM-1-2-HAM
piControl, MPI-ESM1-2-HR
piControl, MPI-ESM1-2-LR
piControl, MRI-ESM2-0
piControl, NorCPM1
piControl, NorESM2-MM
piControl, SAM0-UNICON
piControl, TaiESM1


In [17]:
#rotate grids to [-180,180]
#meshgrid of region of interest
lons_rot = {}
piC_rot = {}

for i in enumerate(models):
    print("Working on "+i[1])
    #rotate longitudes
    mylon0 = lons[i[1]]
    nlon0 = len(mylon0)
    mylon = np.zeros([nlon0])
    zeroind = int((np.abs(mylon0-180.)).argmin())
    n0to180 = nlon0-zeroind+1
    n181to360 = nlon0-n0to180
    mylon[0:nlon0-zeroind] = mylon0[zeroind:]-360.
    mylon[nlon0-zeroind:] = mylon0[0:zeroind]
    lons_rot[i[1]] = mylon
    for ivar in enumerate(myvars):
        myfield0 = dict_piC[i[1]+"_"+ivar[1]]
        myfield = np.zeros(np.shape(myfield0))
        myfield[:,:,0:nlon0-zeroind] = myfield0[:,:,zeroind:]
        myfield[:,:,nlon0-zeroind:] = myfield0[:,:,0:zeroind]
        piC_rot[i[1]+"_"+ivar[1]] = myfield
        print(np.shape(piC_rot[i[1]+"_"+ivar[1]]))

Working on ACCESS-CM2
(6000, 144, 192)
(6000, 144, 192)
Working on ACCESS-ESM1-5
(12000, 145, 192)
(12000, 145, 192)
Working on AWI-CM-1-1-MR
(6000, 192, 384)
(6000, 192, 384)
Working on BCC-CSM2-MR
(7200, 160, 320)
(7200, 160, 320)
Working on BCC-ESM1
(5412, 64, 128)
(5412, 64, 128)
Working on CAMS-CSM1-0
(6000, 160, 320)
(3000, 160, 320)
Working on CAS-ESM2-0
(6600, 128, 256)
(6600, 128, 256)
Working on CESM2
(14400, 192, 288)
(14400, 192, 288)
Working on CESM2-FV2
(6000, 96, 144)
(6000, 96, 144)
Working on CESM2-WACCM
(5988, 192, 288)
(5988, 192, 288)
Working on CESM2-WACCM-FV2
(6000, 96, 144)
(6000, 96, 144)
Working on CMCC-CM2-SR5
(6000, 192, 288)
(6000, 192, 288)
Working on CNRM-CM6-1
(6000, 128, 256)
(6000, 128, 256)
Working on CNRM-CM6-1-HR
(3600, 360, 720)
(3600, 360, 720)
Working on CNRM-ESM2-1
(6000, 128, 256)
(6000, 128, 256)
Working on CanESM5
(12000, 64, 128)
(12000, 64, 128)
Working on E3SM-1-0
(6000, 180, 360)
(6000, 180, 360)
Working on FGOALS-f3-L
(6732, 180, 288)
(67

In [18]:
#interpolation to ROI grid

#define ROI:
tint = 600 #number of months that constitutes one dataset
latmax = 75.
latmin = 30.
lonmin = -80.
lonmax = 30.
lat_rg = np.arange(latmin,latmax+1.)
lon_rg = np.arange(lonmin,lonmax+1.)
lon_rg_mesh,lat_rg_mesh = np.meshgrid(lon_rg,lat_rg)
nlon_rg = len(lon_rg)
nlat_rg = len(lat_rg)
#####******IMPORTANT!  Comment these out if not analyzing hfls
#model_ntime = {}
#model_nint = {} #number of intervals

#regrid loop
piC_rg = {}
for i in enumerate(models):
    mylon = lons_rot[i[1]]
    mylat = lats[i[1]]
    for ivar in enumerate(myvars):
        print("Working on "+i[1]+", "+ivar[1])
        myfield = piC_rot[i[1]+"_"+ivar[1]]
        if ivar[1]=='hfls':
            mymodel_ntime,x,y = np.shape(myfield)
            mymodel_nint = int(np.floor(mymodel_ntime/tint))
            model_ntime[i[1]] = mymodel_ntime
            model_nint[i[1]] = mymodel_nint
        else:
            mymodel_ntime = model_ntime[i[1]]
            mymodel_nint = model_nint[i[1]]
        try:
            myfield_rg = np.zeros([mymodel_ntime,nlat_rg,nlon_rg])
            for itime in range(0,mymodel_ntime):
                myfunc = scipy.interpolate.RectBivariateSpline(mylat,mylon,myfield[itime,:,:])
                myfield_rg[itime,:,:] = myfunc(lat_rg,lon_rg)
            mymodel_nint = model_nint[i[1]]
            myfield_rg_int = np.zeros([mymodel_nint,tint,nlat_rg,nlon_rg])
            for j in range(0,mymodel_nint):
                myfield_rg_int[j,:,:,:] = myfield_rg[j*tint:(j+1)*tint,:,:]
            piC_rg[i[1]+"_"+ivar[1]] = myfield_rg_int
        except:
            print(i[1]+","+ivar[1]+" not available")

Working on ACCESS-CM2, tas
Working on ACCESS-CM2, psl
Working on ACCESS-ESM1-5, tas
Working on ACCESS-ESM1-5, psl
Working on AWI-CM-1-1-MR, tas
Working on AWI-CM-1-1-MR, psl
Working on BCC-CSM2-MR, tas
Working on BCC-CSM2-MR, psl
Working on BCC-ESM1, tas
Working on BCC-ESM1, psl
Working on CAMS-CSM1-0, tas
Working on CAMS-CSM1-0, psl
Working on CAS-ESM2-0, tas
Working on CAS-ESM2-0, psl
Working on CESM2, tas
Working on CESM2, psl
Working on CESM2-FV2, tas
Working on CESM2-FV2, psl
Working on CESM2-WACCM, tas
Working on CESM2-WACCM, psl
Working on CESM2-WACCM-FV2, tas
Working on CESM2-WACCM-FV2, psl
Working on CMCC-CM2-SR5, tas
Working on CMCC-CM2-SR5, psl
Working on CNRM-CM6-1, tas
Working on CNRM-CM6-1, psl
Working on CNRM-CM6-1-HR, tas
Working on CNRM-CM6-1-HR, psl
Working on CNRM-ESM2-1, tas
Working on CNRM-ESM2-1, psl
Working on CanESM5, tas
Working on CanESM5, psl
Working on E3SM-1-0, tas
Working on E3SM-1-0, psl
Working on FGOALS-f3-L, tas
Working on FGOALS-f3-L, psl
Working on F

In [19]:
#apply landmask (maybe? or not?)
#remove seasonal cycle
#detrend
piC_fin_mon = {}

#landmasked version
piC_fin_mon_lm = {}
monthvec = np.linspace(0.,(tint-1)/12,tint)

######**needs to be debugged; this needs to be rotated before regridding!!!! (07/2022)
#create landmask and put it on ROI grid
datadir = "/net/aeolus/aura/hansingh/SOMClimos/"
myfile = "SOM_Control.cam5.0030-0059.ann.nc"
deg2rad = np.pi/180.
Re = 6.4e6
cp = 3850.
rho = 1025.
myData_C = xr.open_dataset(datadir+myfile, decode_times=False)
mylat_cesm = np.array(myData_C.lat)
mylon_cesm = np.array(myData_C.lon)
landfrac = np.array(myData_C.LANDFRAC[0,:,:])
myfunc = scipy.interpolate.RectBivariateSpline(mylat,mylon,landfrac)
landfrac_rg = myfunc(lat_rg,lon_rg)
landfrac_rg = np.where(landfrac_rg<0.5,0.,landfrac_rg)
landfrac_rg = np.where(landfrac_rg>=0.5,1.,landfrac_rg)

for i in enumerate(models):
    mymodel_nint = model_nint[i[1]]
    for ivar in enumerate(myvars):
        print("Working on "+i[1]+", "+ivar[1])
        myfield0 = piC_rg[i[1]+"_"+ivar[1]]
        myfield_mon = np.zeros([mymodel_nint,tint,nlat_rg,nlon_rg])
        #remove seasonal cycle
        for iint in range(0,mymodel_nint):
            for imon in range(0,12):
                monmean = np.mean(myfield0[iint,imon::12,:,:],axis=0)
                myfield_mon[iint,imon::12,:,:] = myfield0[iint,imon::12,:,:]-monmean
        #detrend
        for iint in range(0,mymodel_nint):
            for ilat in range(0,nlat_rg):
                for ilon in range(0,nlon_rg):
                    slope, yint, rval, pval, exerr = scipy.stats.linregress(monthvec,myfield_mon[iint,:,ilat,ilon])
                    myfield_mon[iint,:,ilat,ilon] = myfield_mon[iint,:,ilat,ilon]-monthvec*slope
        #put into dicts
        piC_fin_mon[i[1]+"_"+ivar[1]] = myfield_mon
        #apply ocean mask
        piC_fin_mon_lm[i[1]+"_"+ivar[1]] = myfield_mon*(1.-landfrac_rg)

Working on ACCESS-CM2, tas
Working on ACCESS-CM2, psl
Working on ACCESS-ESM1-5, tas
Working on ACCESS-ESM1-5, psl
Working on AWI-CM-1-1-MR, tas
Working on AWI-CM-1-1-MR, psl
Working on BCC-CSM2-MR, tas
Working on BCC-CSM2-MR, psl
Working on BCC-ESM1, tas
Working on BCC-ESM1, psl
Working on CAMS-CSM1-0, tas
Working on CAMS-CSM1-0, psl
Working on CAS-ESM2-0, tas
Working on CAS-ESM2-0, psl
Working on CESM2, tas
Working on CESM2, psl
Working on CESM2-FV2, tas
Working on CESM2-FV2, psl
Working on CESM2-WACCM, tas
Working on CESM2-WACCM, psl
Working on CESM2-WACCM-FV2, tas
Working on CESM2-WACCM-FV2, psl
Working on CMCC-CM2-SR5, tas
Working on CMCC-CM2-SR5, psl
Working on CNRM-CM6-1, tas
Working on CNRM-CM6-1, psl
Working on CNRM-CM6-1-HR, tas
Working on CNRM-CM6-1-HR, psl
Working on CNRM-ESM2-1, tas
Working on CNRM-ESM2-1, psl
Working on CanESM5, tas
Working on CanESM5, psl
Working on E3SM-1-0, tas
Working on E3SM-1-0, psl
Working on FGOALS-f3-L, tas
Working on FGOALS-f3-L, psl
Working on F

In [20]:
print(model_nint)
tot_model_nint = 0
modelnames_nint = []
for imod in enumerate(models):
    for i in range(0,model_nint[imod[1]]):
        modelnames_nint.append(imod[1])
    tot_model_nint+=model_nint[imod[1]]
print(tot_model_nint)
print(modelnames_nint)

{'ACCESS-CM2': 10, 'ACCESS-ESM1-5': 18, 'AWI-CM-1-1-MR': 10, 'BCC-CSM2-MR': 12, 'BCC-ESM1': 9, 'CAMS-CSM1-0': 5, 'CAS-ESM2-0': 10, 'CESM2': 24, 'CESM2-FV2': 10, 'CESM2-WACCM': 9, 'CESM2-WACCM-FV2': 10, 'CMCC-CM2-SR5': 10, 'CNRM-CM6-1': 10, 'CNRM-CM6-1-HR': 6, 'CNRM-ESM2-1': 10, 'CanESM5': 20, 'E3SM-1-0': 10, 'FGOALS-f3-L': 11, 'FGOALS-g3': 14, 'FIO-ESM-2-0': 11, 'GFDL-CM4': 10, 'GFDL-ESM4': 10, 'GISS-E2-1-G': 17, 'GISS-E2-1-H': 8, 'GISS-E2-2-G': 3, 'HadGEM3-GC31-LL': 10, 'HadGEM3-GC31-MM': 10, 'IITM-ESM': 4, 'INM-CM4-8': 10, 'INM-CM5-0': 24, 'IPSL-CM6A-LR': 5, 'MIROC-ES2L': 10, 'MIROC6': 16, 'MPI-ESM-1-2-HAM': 15, 'MPI-ESM1-2-HR': 10, 'MPI-ESM1-2-LR': 20, 'MRI-ESM2-0': 14, 'NorCPM1': 10, 'NorESM2-MM': 10, 'SAM0-UNICON': 14, 'TaiESM1': 10}
469
['ACCESS-CM2', 'ACCESS-CM2', 'ACCESS-CM2', 'ACCESS-CM2', 'ACCESS-CM2', 'ACCESS-CM2', 'ACCESS-CM2', 'ACCESS-CM2', 'ACCESS-CM2', 'ACCESS-CM2', 'ACCESS-ESM1-5', 'ACCESS-ESM1-5', 'ACCESS-ESM1-5', 'ACCESS-ESM1-5', 'ACCESS-ESM1-5', 'ACCESS-ESM1-5', 'ACC

In [ ]:
for ivar in enumerate(myvars):
    mydata = np.zeros([tot_model_nint,tint,nlat_rg,nlon_rg])
    modtick = 0
    for imod in enumerate(models):
        print("Working on model "+imod[1])
        mymodeln = model_nint[imod[1]]
        mydata[modtick:modtick+mymodeln,:,:,:] = piC_fin_mon[imod[1]+"_"+ivar[1]]
        modtick+=mymodeln
    ds = xr.Dataset({ivar[1]:(("models","t","lat","lon"),mydata)},
                   coords={
                       "models":modelnames_nint,
                       "t": monthvec,
                       "lat": lat_rg,
                       "lon": lon_rg
                          },)
    print("Writing to file")
    ds.to_netcdf("/net/aeolus/aura/hansingh/CMIP6."+ivar[1]+".Control.month.06092022.nc")
    mydata = np.zeros([tot_model_nint,tint,nlat_rg,nlon_rg])
    modtick=0
    for imod in enumerate(models):
        print("Working on model "+imod[1])
        mymodeln = model_nint[imod[1]]
        mydata[modtick:modtick+mymodeln,:,:,:] = piC_fin_mon_lm[imod[1]+"_"+ivar[1]]
        modtick+=mymodeln
    ds = xr.Dataset({ivar[1]:(("models","t","lat","lon"),mydata)},
                   coords={
                       "models":modelnames_nint,
                       "t": monthvec,
                       "lat": lat_rg,
                       "lon": lon_rg
                          },)
    print("Writing to file")
    ds.to_netcdf("/net/aeolus/aura/hansingh/CMIP6."+ivar[1]+".Control.OcnOnly.month.06092022.nc")

Working on model ACCESS-CM2
Working on model ACCESS-ESM1-5
Working on model AWI-CM-1-1-MR
Working on model BCC-CSM2-MR
Working on model BCC-ESM1
Working on model CAMS-CSM1-0
Working on model CAS-ESM2-0
Working on model CESM2
Working on model CESM2-FV2
Working on model CESM2-WACCM
Working on model CESM2-WACCM-FV2
Working on model CMCC-CM2-SR5
Working on model CNRM-CM6-1
Working on model CNRM-CM6-1-HR
Working on model CNRM-ESM2-1
Working on model CanESM5
Working on model E3SM-1-0
Working on model FGOALS-f3-L
Working on model FGOALS-g3
Working on model FIO-ESM-2-0
Working on model GFDL-CM4
Working on model GFDL-ESM4
Working on model GISS-E2-1-G
Working on model GISS-E2-1-H
Working on model GISS-E2-2-G
Working on model HadGEM3-GC31-LL
Working on model HadGEM3-GC31-MM
Working on model IITM-ESM
Working on model INM-CM4-8
Working on model INM-CM5-0
Working on model IPSL-CM6A-LR
Working on model MIROC-ES2L
Working on model MIROC6
Working on model MPI-ESM-1-2-HAM
Working on model MPI-ESM1-2-HR


In [23]:
#Use netcdf files to load dOHU
dict_dOHU = {}
mydir = "/net/aeolus/aura/hansingh/CMIP6AWS_data_processed/"

for i in enumerate(models):
    imod = i[1]
    print("Working on "+imod)
    try:
        OHU_C = xr.open_dataset(mydir+"rsds."+imod+".piControl.nc").rsds-xr.open_dataset(mydir+"rsus."+imod+".piControl.nc").rsus+\
                xr.open_dataset(mydir+"rlds."+imod+".piControl.nc").rlds-xr.open_dataset(mydir+"rlus."+imod+".piControl.nc").rlus-\
                xr.open_dataset(mydir+"hfss."+imod+".piControl.nc").hfss-xr.open_dataset(mydir+"hfls."+imod+".piControl.nc").hfls
        OHU_E = xr.open_dataset(mydir+"rsds."+imod+".abrupt4XCO2.nc").rsds-xr.open_dataset(mydir+"rsus."+imod+".abrupt4XCO2.nc").rsus+\
                xr.open_dataset(mydir+"rlds."+imod+".abrupt4XCO2.nc").rlds-xr.open_dataset(mydir+"rlus."+imod+".abrupt4XCO2.nc").rlus-\
                xr.open_dataset(mydir+"hfss."+imod+".abrupt4XCO2.nc").hfss-xr.open_dataset(mydir+"hfls."+imod+".abrupt4XCO2.nc").hfls
        dict_dOHU[imod] = (OHU_E-OHU_C).squeeze()
    except:
        print("**Model not found.")
        
#Calculate OHU indices
#Calculate Heat Uptake Indices
#Calculate global mean and regional heat uptake
dict_GM_OHU_anom = {}
dict_SHext_OHU_anom = {}
dict_NHext_OHU_anom = {}
deg2rad = np.pi/180.

for i in enumerate(models):
    imod = i[1]
    try:
        mylat = np.array(lats[imod])
        print(imod)
        coslat = np.cos(deg2rad*mylat)
        coslat_SHext = np.where(mylat<-50.,coslat,0.)
        coslat_NHext = np.where(mylat>50.,coslat,0.)
        mydOHU = dict_dOHU[imod]
        dict_GM_OHU_anom[imod] = np.nansum(np.mean(mydOHU, axis=1)*coslat)/np.nansum(coslat)
        dict_SHext_OHU_anom[imod] = np.nansum(np.mean(mydOHU, axis=1)*coslat_SHext)/np.nansum(coslat_SHext)
        dict_NHext_OHU_anom[imod] = np.nansum(np.mean(mydOHU, axis=1)*coslat_NHext)/np.nansum(coslat_NHext)
    except:
        print("**Not found.")

Working on ACCESS-CM2
Working on ACCESS-ESM1-5
Working on AWI-CM-1-1-MR
Working on BCC-CSM2-MR
Working on BCC-ESM1
Working on CAMS-CSM1-0
Working on CAS-ESM2-0
Working on CESM2
Working on CESM2-FV2
Working on CESM2-WACCM
Working on CESM2-WACCM-FV2
Working on CMCC-CM2-SR5
Working on CNRM-CM6-1
Working on CNRM-CM6-1-HR
Working on CNRM-ESM2-1
Working on CanESM5
Working on E3SM-1-0
Working on FGOALS-f3-L
Working on FGOALS-g3
Working on FIO-ESM-2-0
Working on GFDL-CM4
**Model not found.
Working on GFDL-ESM4
Working on GISS-E2-1-G
Working on GISS-E2-1-H
Working on GISS-E2-2-G
Working on HadGEM3-GC31-LL
Working on HadGEM3-GC31-MM
Working on IITM-ESM
Working on INM-CM4-8
Working on INM-CM5-0
Working on IPSL-CM6A-LR
**Model not found.
Working on MIROC-ES2L
Working on MIROC6
Working on MPI-ESM-1-2-HAM
Working on MPI-ESM1-2-HR
Working on MPI-ESM1-2-LR
Working on MRI-ESM2-0
**Model not found.
Working on NorCPM1
Working on NorESM2-MM
Working on SAM0-UNICON
Working on TaiESM1
ACCESS-CM2
ACCESS-ESM1-

In [26]:
print(dict_NHext_OHU_anom)
dOHU_NH = np.zeros([len(modelnames_nint)])
for i in enumerate(modelnames_nint):
    try:
        dOHU_NH[i[0]] = dict_NHext_OHU_anom[i[1]]
    except:
        dOHU_NH[i[0]] = np.nan
        
print(dOHU_NH)
myds = xr.Dataset({"dOHU":(("models"),dOHU_NH)},
                  coords = {
                      "models": modelnames_nint
                  },)
#os.remove("/net/aeolus/aura/hansingh/CMIP6.OcnHeatUptake.4XCO2.062022.pdf")
myds.to_netcdf("/net/aeolus/aura/hansingh/CMIP6_OHUdata/CMIP6.OcnHeatUptake.4XCO2.06092022.nc")
print(myds)

{'ACCESS-CM2': 5.069753901720349, 'ACCESS-ESM1-5': 2.4882552462510663, 'AWI-CM-1-1-MR': 1.8462341228047452, 'BCC-CSM2-MR': 3.9593138219070494, 'BCC-ESM1': 4.44237720491568, 'CAMS-CSM1-0': 1.4875264065295346, 'CAS-ESM2-0': 2.165541990439814, 'CESM2': 8.839188644119957, 'CESM2-FV2': 9.16785190771157, 'CESM2-WACCM': 8.535441704125132, 'CESM2-WACCM-FV2': 8.869834498690738, 'CMCC-CM2-SR5': 3.1998359069193083, 'CNRM-CM6-1': 5.954116590272326, 'CNRM-CM6-1-HR': 7.1284849415439595, 'CNRM-ESM2-1': 5.2874700756529736, 'CanESM5': 4.5455000859568395, 'E3SM-1-0': 3.7530974761661042, 'FGOALS-f3-L': 0.997110058880613, 'FGOALS-g3': 4.500549003077796, 'FIO-ESM-2-0': 9.52320467207591, 'GFDL-ESM4': 5.522209060276806, 'GISS-E2-1-G': 10.009460452837633, 'GISS-E2-1-H': 5.464185275977079, 'GISS-E2-2-G': 7.181215819251269, 'HadGEM3-GC31-LL': 5.428590548651897, 'HadGEM3-GC31-MM': 6.533046645544147, 'IITM-ESM': 6.125721652814539, 'INM-CM4-8': 2.035252762553003, 'INM-CM5-0': 1.5399693180392517, 'MIROC-ES2L': 5.09